In [80]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
import csv

In [81]:
def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)

In [82]:
import datetime
def rename_files(data_dir):
    dates_set = set()

    for filename in pathlib.Path(data_dir).glob('*.csv'):
        if "Product Comment Report" in filename.name:
            continue
        with open(filename, 'r') as csv_file:
            rows = csv.reader(csv_file, delimiter=',')
            headers = next(rows)
            sheet = {col: [] for col in headers}
            for row in rows:
                for i, col in enumerate(headers):
                    sheet[col].append(row[i])
        if not sheet['1_resolve_time']:
            continue
        date_str = sheet['1_resolve_time'][1].split()[0]
        dates_set.add(date_str)

        title = sheet['title'][0].strip()

    for date in dates_set:
        start_date, end_date = date, date

        for filename in pathlib.Path(data_dir).glob('*.csv'):
            if "Product Comment Report" in filename.name:
                continue
            with open(filename, 'r') as csv_file:
                rows = csv.reader(csv_file, delimiter=',')
                headers = next(rows)
                sheet = {col: [] for col in headers}
                for row in rows:
                    for i, col in enumerate(headers):
                        sheet[col].append(row[i])
            if not sheet['1_resolve_time'] or len(sheet['1_resolve_time']) < 2:
                continue
            date_list = []
            for date_str in sheet['1_resolve_time']:
                date = datetime.datetime.strptime(date_str.split()[0], '%Y-%m-%d').date()
                date_list.append(date)

            date_list.sort(reverse=True)
            start_date = date_list[-1].strftime('%Y-%m-%d')
            end_date = date_list[0].strftime('%Y-%m-%d')

            new_file_name = f"{title}_{start_date}_{end_date}.csv"
            new_filepath = os.path.join(data_dir, new_file_name)
            
            if os.path.exists(new_filepath):
                os.remove(new_filepath)
            
            os.rename(filename, new_filepath)


rename_files('E:/tt/qa_daily_automation/Product comment report/R1')
rename_files('E:/tt/qa_daily_automation/Product comment report/R2')


In [83]:
#Supporting Dashboard
#Queue
queue_list = pd.read_excel('E:/tt/qa_daily_automation/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

#Variable
Date_variable = pd.read_excel('E:/tt/qa_daily_automation/variable.xlsx')
from datetime import datetime
Date_variable['Start date'] = Date_variable['Start date'].apply(convert_datetime)
Date_variable['End date'] = Date_variable['End date'].apply(convert_datetime)

Start_Date_False_PCR = datetime.strptime(str(Date_variable.loc[6, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_False_PCR = datetime.strptime(str(Date_variable.loc[6, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_Diff_PCR = datetime.strptime(str(Date_variable.loc[5, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Diff_PRC = datetime.strptime(str(Date_variable.loc[5, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Date_variable

,Unnamed: 0,Start date,End date
0,appeal_diff_case,2023-08-27,2023-08-27
1,appeal_dailyreport,2023-08-26,2023-08-26
2,regular_diff_case,2023-08-21,2023-08-27
3,regular_false_case,2023-08-26,2023-08-26
4,rock_dailyreport,2023-08-26,2023-08-26
5,PCR_diff_case,2023-08-21,2023-08-27
6,PCR_false_case,2023-08-26,2023-08-26
7,sideprj_shoptab,2023-08-27,2023-08-27
8,sideprj_ugads,2023-08-27,2023-08-27
9,sideprj_dailyreport,2023-08-26,2023-08-26


In [84]:
def merge_csv_files(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            # Đặt kiểu dữ liệu của cột có chuỗi dài 19 số thành 'object'
            df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    # Ghép tất cả các dataframe trong danh sách dfs
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df


In [85]:
qa_rawdata = merge_csv_files('E:/tt/qa_daily_automation/Product comment report/R2')
mod_rawdata = merge_csv_files('E:/tt/qa_daily_automation/Product comment report/R1')

In [86]:
mod_rawdata['title'] = mod_rawdata['title'].replace('VN LL Product Comment Report', 'VN Product Comment Report')
mod_rawdata['title'].unique()

array(['VN Product Comment Report'], dtype=object)

In [87]:
qa_rawdata['title'].value_counts()

title
QA VN LL Product Comment Report    24879
Name: count, dtype: int64

In [88]:
qa_rawdata = pd.merge(qa_rawdata,queue_list,how='left',left_on='title',right_on='QA QUEUE NAME')

In [89]:
merged_df = pd.merge(qa_rawdata, mod_rawdata, how='left', 
                     left_on=['MOD QUEUE NAME','object_id'], 
                     right_on=['title','object_id'],
                     suffixes=('_qa', '_mod'))
merged_df.dropna(subset='1_rejectLabel_mod',inplace=True)

In [90]:
merged_df.columns

Index(['task_id_qa', 'project_id_qa', 'title_qa', 'object_id', 'status_qa',
       'create_time_qa', 'closed_time_qa', '1_verifier_qa', '1_assign_time_qa',
       '1_resolve_time_qa', '1_duration_qa', '1_verify_data_qa',
       '1_auditResult_qa', '1_auditStatus_qa', '1_rejectLabel_qa',
       'tcs_link_qa', 'is_simulation_qa', 'Unnamed: 17_qa', 'Unnamed: 18_qa',
       'Unnamed: 19_qa', 'BUSINESS', 'COMPOUND', 'CATEGORY', 'MOD QUEUE ID',
       'MOD QUEUE NAME', 'QA QUEUE ID', 'QA QUEUE NAME', 'QUEUE GROUP',
       'STATUS', 'Priority', 'Latency', 'AHT (secs)', 'Policy_type',
       'Classify', 'task_id_mod', 'project_id_mod', 'title_mod', 'status_mod',
       'create_time_mod', 'closed_time_mod', '1_verifier_mod',
       '1_assign_time_mod', '1_resolve_time_mod', '1_duration_mod',
       '1_verify_data_mod', '1_auditResult_mod', '1_auditStatus_mod',
       '1_rejectLabel_mod', 'tcs_link_mod', 'is_simulation_mod',
       'Unnamed: 17_mod', 'Unnamed: 18_mod', 'Unnamed: 19_mod'],
      

In [91]:
merged_df[['Moderation Result','Respondent Result']] = merged_df[['1_verify_data_mod','1_verifier_qa']].replace(to_replace=r'^auditStatus: ',value= "Audit Result:: ",regex=True).astype(str)
pcr_result = ['remove', 'normal', 'selfvisible']
columns = ['Moderation Result', 'Respondent Result']

def match_and_concatenate(cell):
    cell = str(cell).lower()
    cell = cell.replace('_x000d_', '')
    found_words = []
    audit_exp = re.search(r'(.*auditreason:)(.*)(\n|$)', cell)
    reject_exp = re.search(r'(.*rejectlabel:)(.*)(\n|$)', cell)
    if audit_exp:
        cell_split = audit_exp.group(2).strip()
    elif reject_exp:
        cell_split = reject_exp.group(2).strip()
    else:
        cell_split = cell
    cell_split = re.sub(r'[\[\]"]', '', cell_split)
    matches = re.findall('|'.join(pcr_result), cell_split, re.IGNORECASE)
    found_words.extend(matches)
    return ",".join(found_words) if found_words else None
for column in ['Moderation Result', 'Respondent Result']:
    merged_df.loc[merged_df['title_qa'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), column] = merged_df.loc[merged_df['title_qa'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), column].apply(match_and_concatenate)
    
merged_df['Mod Result'] = merged_df['1_rejectLabel_mod'].apply(match_and_concatenate)
merged_df['BPO QA Result'] = merged_df['1_rejectLabel_qa'].apply(match_and_concatenate)

In [92]:
merged_df = merged_df[['title_qa','project_id_qa','1_verifier_mod','1_resolve_time_mod',
                       'object_id','Mod Result','1_auditResult_mod','1_duration_mod',
                       '1_verifier_qa','1_resolve_time_qa','task_id_qa','tcs_link_qa',
                       'BPO QA Result','1_auditResult_qa']].rename(columns={'title_qa':'Sampling Queue','project_id_qa':'Queue ID','1_verifier_mod':'Moderator name','1_resolve_time_mod':'Moderation time','object_id':'Object ID','1_auditResult_mod':'Mod Reason',
                                                                           '1_duration_mod':'Mod AHT','1_verifier_qa':'BPO QA','1_resolve_time_qa':'BPO QA Date','task_id_qa':'BPO QA Task ID','tcs_link_qa':'BPO QA Link','1_auditResult_qa':'BPO QA Reason'})

In [93]:
merged_df['Object ID'] = "id=" + merged_df['Object ID'].astype(str)
merged_df['Queue ID'] = "id=" + merged_df['Queue ID'].astype(str)
merged_df['BPO QA Task ID'] = "id=" + merged_df['BPO QA Task ID'].astype(str)

In [94]:
Policy_errors_1 = [
    'prohibited products','unsupported products','restricted products','adult nudity and sexual activities','hateful behaviors','minor safety','absolute terms',
    'malicious comparison','title spam','abnormal price','inconsistent information','other languages','misleading functionality and effect','redirect traffic',
    'missing brand authorization','unfair trading','obscuring product images','target to minors','underage commercial behavior','false promise','false promises','sympathy promotion',
    'misleading size/weight/quantity','misleading patent/endorsement/metrics/awards','misleading origin','misleading return & refund service','misleading guarantees and warranties',
    'irrelevant promotion','give away','reproduced content','soliciting mutual interactions','still frame','possible pirated contents','possible pirated content',
    'non product promotion content','vulgar behavior','disturbing animal body','insulting and abusive content','body impurities','dirty environment','eye-catching sensitive content',
    'bind a sell b - prohibited product','misleading logistics','misleading brand','brand circumvention','copycat products','violence',
    'illegal behavior','hatespeech and harassment','possible ip infringement','spam','invalid company name','inaccurate company name','inaccurate company address','missing identity document',
    'expired documentation','black-and-white copy','unclear documentation','invalid watermark or logo','incomplete documentation','inaccurate representative name','inaccurate id number',
    'inaccurate documentation','inaccurate tax name','inaccurate tax number','inaccurate brand name','inaccurate trademark name','inaccurate trademark number','inaccurate application date',
    'inaccurate expiry date','invalid trademark','inaccurate registrant','unqualified destination market','inaccurate class(es) of goods/service','unqualified owner','inaccurate trademark certificate',
    'expired trademark certificate','unclear trademark certificate','inauthentic trademark certificate','unqualified 1st level distributor','inaccurate 1st level distributor letter',
    'expired 1st level distributor letter','unclear 1st level distributor letter','inauthentic 1st level distributor letter','unqualified 2nd level distributor',
    'inaccurate 2nd level distributor letter','expired 2nd level distributor letter','unclear 2nd level distributor letter','inauthentic 2nd level distributor letter',
    'unqualified third level distributor','inaccurate third level distributor letter','expired third level distributor letter','unclear third level distributor letter',
    'inauthentic third level distributor letter','missing buyer information','missing quantity of purchase item','missing purchase item information','missing purchase date',
    'unable to locate brand item product','purchase date mismatch','seller information mismatch','missing price of purchase item','missing seller information',
    'product quantity insufficient','exceeded validity period','altered document','lack of supportive information','unable to recognise source of purchase','underage applicant',
    'gibberish shop name','abnormal company status','traffic redirection','inaccurate company number','oversea personal seller','ps tampering trace','inauthentic documentation',
    'violent extremism','hate speech','unauthorized brand information','impersonation','suicide and dangerous acts','illegal activities and regulated goods',
    'harassment and bullying','child endangerment information','counterfeit product information','inauthentic certification of incorporation','inaccurate register date','erotic nudity','hateful contexts','partner name or logo relates to ''tiktok''','inauthentic work environment','repeated image','inappropriate watermark or logo',
    'incomplete information','inaccurate and misleading review','slightly vulgar','inaccurate package weight information','inappropriate speech','malicious behaviors','vulgar issues',
    'imagery depicting skin defects','imagery depicting indecent body organs','imagery depicting indecent teeth&oral','imagery depicting animal meat/carcasses',
    'imagery depicting unpleasant animal or insect','imagery depicting impurities','horror content','scene of squalor','offensive content','human excrement','mental illness content',
    'unsuitable for recommendation','surprise-based product','vat number check','slide show','counterfeit word indication','brand never produced','bind a sell b-prohibited product',
    'pirated content- dedup recall','pirated content','possible counterfeit product','click bait','click-bait','non shippable products','physical trademark infringement product','incorrect category high risk','low cost content',  
    'gambling related behaviour','information trademark infringement product','physically damaged id','suspicious brand promotion'
    ]
Policy_errors_2 = list(map(lambda x: x.replace(" ", "_") + "0005",Policy_errors_1))

Policy_errors = Policy_errors_1 + Policy_errors_2

merged_df['Mod Reason'] = merged_df['Mod Reason'].str.lower()
merged_df['BPO QA Reason'] = merged_df['BPO QA Reason'].str.lower()

Policy_errors_pattern = "|".join(Policy_errors )

def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = np.nan
    return return_str

merged_df['Mod Reason'] = merged_df['Mod Reason'].astype(str).apply(lambda y: pattern_searcher(search_str=y, search_list=Policy_errors_pattern))
merged_df['BPO QA Reason'] = merged_df['BPO QA Reason'].astype(str).apply(lambda z: pattern_searcher(search_str=z, search_list=Policy_errors_pattern))

def d(row):
    last_index = row.last_valid_index()
    return row[last_index] if last_index else np.nan

#Final Policy Errors
merged_df['Final policy errors'] = merged_df[['Mod Reason','BPO QA Reason']].apply(d, axis=1)
merged_df.shape

(95906, 15)

In [95]:
#Add diff case

merged_df.loc[merged_df['Mod Result'] != merged_df['BPO QA Result'],'Diff case']="FALSE"
merged_df.loc[merged_df['Diff case'].isnull(),'Diff case']="TRUE"


In [96]:
merged_df['BPO QA Date'] = pd.to_datetime(merged_df['BPO QA Date'])
merged_df['Moderation time'] = pd.to_datetime(merged_df['Moderation time'])

merged_df['QA Week'] = merged_df['BPO QA Date'].dt.isocalendar().week - (merged_df['BPO QA Date'].dt.weekday == 0).astype(int)
merged_df['Mod Week'] = merged_df['Moderation time'].dt.isocalendar().week - (merged_df['Moderation time'].dt.weekday == 0).astype(int)

In [97]:
Final_false_case = merged_df[(merged_df['BPO QA Date'] >= merged_df['Moderation time']) &
                        (merged_df['QA Week'] >= merged_df['Mod Week'])]

Final_false_case = Final_false_case.drop(columns=['QA Week', 'Mod Week'])
Final_false_case = Final_false_case.sort_values(by='BPO QA Date', ascending=True).drop_duplicates(subset='BPO QA Task ID', keep='last')

In [98]:
def filter_time( df,  from_date, to_date):

    Final_false_case = df[(df['BPO QA Date'].dt.strftime('%Y-%m-%d') >= from_date) & (df['BPO QA Date'].dt.strftime('%Y-%m-%d') <= to_date)]

    #Final_false_case.to_excel(f"C:/Users/v6210227/Desktop/Product comment {from_date}{to_date}.xlsx", index=False)

    return Final_false_case
# Fill in moderation time (Tue -> Today - 1)
Regular_diff_pcr = filter_time(Final_false_case, Start_Date_Diff_PCR, End_Date_Diff_PRC)

Regular_diff_case_pcr = Regular_diff_pcr[Regular_diff_pcr['Diff case']=="FALSE"]
Regular_diff_case_pcr

,Sampling Queue,Queue ID,Moderator name,Moderation time,Object ID,Mod Result,Mod Reason,Mod AHT,BPO QA,BPO QA Date,BPO QA Task ID,BPO QA Link,BPO QA Result,BPO QA Reason,Final policy errors,Diff case
98328,QA VN LL Product Comment Report,id=7181739508772110849,tran.tl@trans-cosmos.com.vn,2023-08-24 16:56:03,id=7270790179282059525,"normal,normal,normal,normal,normal,remove",harassment and bullying,47.0,my.ntk@trans-cosmos.com.vn,2023-08-25 09:55:27,id=7270998066709266946,https://tcs-sg.bytelemon.com/worktable/7181739...,"normal,normal,normal,normal,remove,remove",harassment and bullying,harassment and bullying,FALSE
98754,QA VN LL Product Comment Report,id=7181739508772110849,sang.vtn@trans-cosmos.com.vn,2023-08-25 21:10:13,id=7271261314017313029,"normal,normal,selfvisible",redirect traffic,32.0,my.ntk@trans-cosmos.com.vn,2023-08-26 11:12:15,id=7271340683640799746,https://tcs-sg.bytelemon.com/worktable/7181739...,"normal,normal,normal",NaN,redirect traffic,FALSE
99121,QA VN LL Product Comment Report,id=7181739508772110849,vy.dtt3@trans-cosmos.com.vn,2023-08-26 15:41:14,id=7271543913712763142,remove,harassment and bullying,8.0,tram.nn@trans-cosmos.com.vn,2023-08-27 20:31:20,id=7271754387612172801,https://tcs-sg.bytelemon.com/worktable/7181739...,normal,NaN,harassment and bullying,FALSE


In [99]:
Regular_diff_pcr['BPO QA Date']

81177   2023-08-21 08:06:48
91771   2023-08-21 11:05:29
81936   2023-08-21 11:05:32
81945   2023-08-21 11:06:34
81203   2023-08-21 11:06:39
                ...        
99233   2023-08-27 20:38:32
99235   2023-08-27 20:38:36
99237   2023-08-27 20:38:39
99239   2023-08-27 20:38:46
99241   2023-08-27 20:38:51
Name: BPO QA Date, Length: 798, dtype: datetime64[ns]

##### Diff case

In [100]:
def filter_time( df,  from_date, to_date):

    Diff_case_PCR = df[(df['Moderation time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['Moderation time'].dt.strftime('%Y-%m-%d') <= to_date)]
    Diff_case_PCR['Moderation time'] = pd.to_datetime(Diff_case_PCR['Moderation time'].apply(convert_date))
    Diff_case_PCR = Diff_case_PCR[Diff_case_PCR['Diff case'] == 'FALSE']
    Diff_case_PCR = Diff_case_PCR.groupby(by=['Moderation time','Sampling Queue','Moderator name'], as_index=False).agg({'BPO QA Task ID':'count'})
    Diff_case_PCR = Diff_case_PCR[['Moderation time','Sampling Queue','Moderator name','BPO QA Task ID']]
    Diff_case_PCR.rename(columns={'BPO QA Task ID':'No. Diff cases'}, inplace=True)
   
    return Diff_case_PCR, from_date, to_date
# Fill in moderation time (Tue -> Today - 1)
Final_False_Case_PCR, Start_date_PCR, End_date_PCR = filter_time(Regular_diff_pcr, Start_Date_False_PCR, End_Date_False_PCR)

Final_False_Case_PCR.to_excel('E:/tt/qa_daily_automation/Product comment report.xlsx')

C:\Users\anh.td\AppData\Local\Temp\ipykernel_26720\4193611911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Diff_case_PCR['Moderation time'] = pd.to_datetime(Diff_case_PCR['Moderation time'].apply(convert_date))
